In [1]:
import sqlalchemy as db
import pandas as pd
import os
import statsmodels.api as sm
import numpy as np
from scipy import stats

data_path  = r'C:\Users\Lazar\Desktop\Financial Volatility'

                        
data_path  = r'C:\Users\Lazar\Desktop\Financial Volatility\Assignment'

# create connection to sqlite database
db_path = os.path.join(data_path, 'database.db')
db_engine = db.create_engine('sqlite:///' + db_path)

# select your desired table 
data = pd.read_sql("select *from returns", db_engine)

def gamma(x,h):
  n=len(x)
  h=abs(h)
  x = x-x.mean()
  gy = np.multiply(np.asarray(x[0:(n-h)]),x[h:(n)])
  gamma= (gy.sum()).item()/n
  return gamma

def rho(x,h):
  rho = gamma(x,h)/gamma(x,0)
  return rho

def gamma_asy(x,h):
    '''

    Parameters
    ----------
    x : pd.Series
        Time series of stock return.
    h : nummeric.
        Number of lags to be included in the matrix. For example, 5.
    Returns
    -------
    Assymetric cov.matrix of the sample autocorrelation.

    '''
    n = len(x)
    h = abs(h)
    x = x - x.mean()
    x2 = x**2
    gamma_asy = np.empty((h,h))
    gamma_asy[:] = np.nan
    for i in (range(0,h)):
        for j in (range(i,h)):
            gamma_asy[i,j] = np.multiply(np.asarray(x[(j-i):(n-i-1)]),np.multiply(np.asarray(x[0:(n-j-1)]),np.asarray(x2[(j+1):(n)]))).sum()/n
            gamma_asy[j,i] = gamma_asy[i,j] 
    rho_asy = np.multiply((1/gamma(x,0)**2),gamma_asy)
    return gamma_asy, rho_asy

def corr_Box_test(x,h, print_res = True):
    '''

    Parameters
    ----------
    x : pd.Series
        DESCRIPTION.
    h : nummeric
        Number of lags to be included in the matrix. For example, 5.
    print_res : Bool.
        Specifies whether results should be printed.

    Returns
    -------
     Corrected portmanteau test under GARCH assumption for the hth lag.

    '''

    n = len(x)
    a = gamma_asy(x,h)
    acf_val = np.empty((1,h))
    acf_val[:] = np.nan
    for i in range(0,h):
        acf_val[0,i] = rho(x,i+1)
    val = n*np.dot(np.dot(acf_val, np.linalg.inv(a[1])), acf_val.T)
    if print_res is True:
        print(val, (1 - stats.chi2.cdf(val, h)))
    return(val, (1 - stats.chi2.cdf(val, h)))



for i in range(1,21):
    corr_Box_test(data['EEM'], i)

[[1.34637406]] [[0.24591301]]
[[2.20586545]] [[0.33189629]]
[[2.9902766]] [[0.39312676]]
[[5.22225079]] [[0.26524379]]
[[7.15276584]] [[0.20952459]]
[[7.25475423]] [[0.29792898]]
[[7.30251654]] [[0.39807604]]
[[10.42068238]] [[0.23673152]]
[[10.4532965]] [[0.31504883]]
[[11.33655952]] [[0.33190502]]
[[11.338088]] [[0.41538937]]
[[12.01790667]] [[0.4442426]]
[[12.93619413]] [[0.45275224]]
[[13.18730886]] [[0.51183067]]
[[13.18749225]] [[0.58781869]]
[[15.83184182]] [[0.46475748]]
[[18.18203503]] [[0.37744482]]
[[20.04937145]] [[0.33004696]]
[[20.0633811]] [[0.39077336]]
[[20.25793285]] [[0.44190228]]
